<a href="https://colab.research.google.com/github/vilsonrodrigues/PortfolioDataScience/blob/master/Challange_ITA/Challange_ITA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Challange ITA 2021**

## 1 Import libraries

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 2 Data Understanding

### 2.0 Reading cvs files

Modelo de submissão da resposta ao server do ITA

In [214]:
submission = pd.read_csv('http://www.comp.ita.br/~verri/dsc/edicoes/2021/submission-sample.csv')
submission.head(2)

cent_price_cor  cent_trans_cor
0               0               0
1               0               0

In [215]:
df_train = pd.read_csv('http://www.comp.ita.br/~verri/dsc/edicoes/2021/public-train.csv')
df_train.head(2)

n    p    f   x   y  ...   l4   e1   e2  cent_price_cor  cent_trans_cor
0  709  0.7  0.2  16  12  ...  1.6  1.5  2.6          -0.169           0.375
1  921  0.6  0.3  19  17  ...  1.8  0.6  0.4          -0.075           0.234

[2 rows x 28 columns]

In [216]:
df_test = pd.read_csv('http://www.comp.ita.br/~verri/dsc/edicoes/2021/public-test.csv')
df_test.head(2)

id    n    p    f   x   y  z  ...   g2     l1   l2     l3   l4   e1   e2
0   1  558  0.5  0.2  28  21  2  ...  1.8  0.149  2.0  0.078  1.7  0.5  2.9
1   2  910  0.9  0.3  28  19  9  ...  1.0  0.086  0.8  0.030  0.8  0.2  1.6

[2 rows x 27 columns]

###  2.1 Data Description

**Features**

* **n**: Indicador do número de traders/players no sistema. São os consumidores que estão interessados naquele espaço aéreo. Eles podem adquirir ou vendê-los, fazendo uso de especulação.

* **p**: Fração de traders.

* **f**: Indicador do interesse desses traders/players em relação as permissões.

* **x**, **y**, **z**: Dimensões do espaço aéreo.

* **a1** e **a2**: Média e desvio padrão do coeficiente do preço fundamental dos consumidores.

* **a3** e **a4**: O mesmo para os traders.
 
* **b1** e **b2**: Média e desvio padrão do coeficiente do preço de mercado dos consumidores.

* **b3** e **b4**: O mesmo para os traders.

* **c1**, **c2**: Média e desvio padrão do coeficiente do preço aleatorio dos consumidores.

* **c3** e **c4**: O mesmo para os traders.
 
* **g1** e **g2**: Média e desvio padrão do grau de agressividade dos consumidores

 * **l1**, **l2**: Média e dp do coeficiente de desvalorização para os consumidores

 * **l3** e **l4**: O mesmo para os traders.

* **e1** e **e2**: Variabilidade no preço fundamental dos consumidores e traders; respectivamente


b1 e g1

a1 = prejudicial pro modelo

In [157]:
df_train = df_train.drop(columns=['a2','a4','b2','b4','c2','l2','l4','n','x','y','z','f','p','f','c4','e1','e2','l3','l1','a1','g2','a3','c3','b3','c1','g1'])

In [158]:
df_test = df_test.drop(columns=['a2','a4','b2','b4','c2','l2','l4','n','x','y','z','f','p','f','c4','e1','e2','l3','l1','a1','g2','a3','c3','b3','c1','g1'])

In [175]:
df_train.head()

n    p    f   x   y  ...   e1   e2  cent_price_cor  cent_trans_cor  (b1-a1)/b2
0  709  0.7  0.2  16  12  ...  1.5  2.6          -0.169           0.375  -12.000000
1  921  0.6  0.3  19  17  ...  0.6  0.4          -0.075           0.234  -10.000000
2  177  0.8  0.4  14  12  ...  2.7  0.3          -0.177           0.389   -2.916667
3  415  0.7  0.5  24  11  ...  0.5  0.1          -0.102           0.358   11.250000
4  802  0.8  0.4  21  10  ...  2.0  0.6          -0.034           0.180   -7.857143

[5 rows x 29 columns]

In [194]:
df_train['b1/e1'] = df_train['b1'] / df_train['e1']
df_test['b1/e1'] = df_test['b1'] / df_test['e1']

In [ ]:
df_train['(c1-c3)/c2'] = (df_train['c1'] - df_train['c3'])/df_train['c2']
df_test['(c1-c3)/c2'] = (df_test['c1'] - df_test['c3'])/df_test['c2']

In [ ]:
df_train['(c1-c3)/c4'] = (df_train['c1'] - df_train['c3'])/df_train['c4']
df_test['(c1-c3)/c4'] = (df_test['c1'] - df_test['c3'])/df_test['c4']

In [ ]:
df_train['(a1-b1)/b2'] = (df_train['a1'] - df_train['b1'])/df_train['b2']
df_test['(a1-b1)/b2'] = (df_test['a1'] - df_test['b1'])/df_test['b2']

In [ ]:
df_train['(a1-b1)/a2'] = (df_train['a1'] - df_train['b1'])/df_train['a2']
df_test['(a1-b1)/a2'] = (df_test['a1'] - df_test['b1'])/df_test['a2']

In [ ]:
df_train['g1/a1'] = df_train['g1']/df_train['a1'] 
df_test['g1/a1'] = df_test['g1']/df_test['a1'] 

In [ ]:
df_train['(c1*a1)/(1-b1)'] = (df_train['c1'] * df_train['a1'])/(1.0 - df_train['b1'])
df_test['(c1*a1)/(1-b1)'] = (df_test['c1'] * df_test['a1'])/(1.0 - df_test['b1'])

In [ ]:
df_train['(b1*a1)/c1'] = (df_train['b1'] * df_train['a1'])/df_train['c1']
df_test['(b1*a1)/c1'] = (df_test['b1'] * df_test['a1'])/df_test['c1']

In [ ]:
df_train['c1*a1*b1'] = df_train['c1'] * df_train['a1'] * df_train['b1']
df_test['c1*a1*b1'] = df_test['c1'] * df_test['a1'] * df_test['b1']

In [ ]:
df_train['c1/c2'] = df_train['c1'] / df_train['c2']
df_test['c1/c2'] = df_test['c1'] / df_test['c2']

In [ ]:
df_train['c3/c4'] = df_train['c3'] / df_train['c4']
df_test['c3/c4'] = df_test['c3'] / df_test['c4']

In [ ]:
df_train['b1-a1'] = df_train['b1'] - df_train['a1']
df_test['b1-a1'] = df_test['b1'] - df_test['a1']

In [174]:
#olhar com mais carinho
df_train['(b1-a1)/b2'] = (df_train['b1'] - df_train['a1'])/df_train['b2']
df_test['(b1-a1)/b2'] = (df_test['b1'] - df_test['a1'])/df_test['b2']

In [ ]:
#olhar com mais carinho
df_train['(a1-b3)/b4'] = (df_train['a1'] - df_train['b3'])/df_train['b4']
df_test['(a1-b3)/b4'] = (df_test['a1'] - df_test['b3'])/df_test['b4']

In [ ]:
df_train['l1*g1'] = df_train['l1'] * df_train['g1']
df_test['l1*g1'] = df_test['l1'] * df_test['g1']

In [ ]:
df_train['l1*f'] = df_train['l1'] * df_train['f']
df_test['l1*f'] = df_test['l1'] * df_test['f']

In [ ]:
#df_train['a3*f*p'] = df_train['a3'] * df_train['f'] * df_train['p']
#df_test['a3*f*p'] = df_test['a3'] * df_test['f'] * df_test['p']

In [ ]:
df_train['a3*f'] = df_train['a3'] * df_train['f'] 
df_test['a3*f'] = df_test['a3'] * df_test['f'] 

In [ ]:
df_train['b3*f'] = df_train['b3'] * df_train['f'] 
df_test['b3*f'] = df_test['b3'] * df_test['f'] 

In [ ]:
df_train['aaa'] = (df_train['e1'] - df_train['e2']) * df_train['f']

In [ ]:
df_train['(b3-c1)/c2'] = (df_train['b3'] - df_train['c1']) / df_train['c2']

In [ ]:
df_train.head(2)

media do coeficiente do preço fundamental dos traders, dividido, pelo produto da variabilidade no preço fundamental e media do coeficiente do preço aleatorio dos consumidores

In [ ]:
df_train['a3'][0] / (df_train['e2'][0] *df_train['c3'][0])

produto do numero de players e indicador de interesse. (nf)

In [ ]:
df_train['n'][0] * df_train['f'][0]

numero de traders

In [ ]:
df_train['n'][0]/df_train['e2'][0]

In [ ]:
#df_train['nf'] = (df_train['n'] * df_train['f'])

In [ ]:
#df_train['a3-nf'] = df_train['a3'] / (df_train['n'] * df_train['f'])

In [ ]:
df_train['xyz'] =  df_train['x']* df_train['y']* df_train['z']

In [ ]:
df_train['a1-g1'] = df_train['a1'] / df_train['g1']

Melhor até agora

In [ ]:
df_train['(a1-b1)/b2'] = (df_train['a1'] - df_train['b1']) / df_train['b2']

**Targets**

* **cent_price_cor**: Correlação entre preço final e centralidade das permissões de vôo.

* **cent_trans_cor**: Idem para o número de transações.

**A correlação usada foi a de person**

Antes


* **a*** e **e***: Coeficientes do comportamento dos agentes (referente a opinião particular do preço). Quanto maior for, mais o agente vai respeitar a opinião.

* **b***: Coeficientes do comportamento dos agentes (referente ao valor de mercado). Quanto maior o b mais os agentes olham o valor das últimas vendas para definir seu lance.

* **c***: Coeficientes do comportamento dos agentes (referente ao comportamento aleatório). Coeficiente de aleatoriedade. Quanto maior, mais os players jogam para cima ou para baixo os lances.

* **g***: Coeficientes referentes a agressividade de compra dos agentes consumidores. Se o G for muito grande e ele não conseguir comprar, ele começa a realizar lances cada vez mais altos. Com G pequeno, é uma indicação que os players estão mais conservadores; melhor não conseguir comprar todo o espaço do que jogar o preço muito alto.

* **l***: Coeficientes de desvalorização do preço das permissões. Perto do vencimento do espaço aéreo o player necessita revender para não perder dinheiro.

* **size***: Tamanho do ambiente (número de permissões a serem negociadas).

* **altitute**: Altura do ambiente. Até onde os drones poderam voar.

* **agents**: Número de agentes nessa simulação.

### 2.2 Data Quality

Sem dados faltantes. Sem nenhum atributo categórico

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11940 entries, 0 to 11939
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   n               11940 non-null  int64  
 1   p               11940 non-null  float64
 2   f               11940 non-null  float64
 3   x               11940 non-null  int64  
 4   y               11940 non-null  int64  
 5   z               11940 non-null  int64  
 6   a1              11940 non-null  float64
 7   a2              11940 non-null  float64
 8   a3              11940 non-null  float64
 9   a4              11940 non-null  float64
 10  b1              11940 non-null  float64
 11  b2              11940 non-null  float64
 12  b3              11940 non-null  float64
 13  b4              11940 non-null  float64
 14  c1              11940 non-null  float64
 15  c2              11940 non-null  float64
 16  c3              11940 non-null  float64
 17  c4              11940 non-null 

O mesmo ocorre nos dados de teste

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8059 entries, 0 to 8058
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      8059 non-null   int64  
 1   n       8059 non-null   int64  
 2   p       8059 non-null   float64
 3   f       8059 non-null   float64
 4   x       8059 non-null   int64  
 5   y       8059 non-null   int64  
 6   z       8059 non-null   int64  
 7   a1      8059 non-null   float64
 8   a2      8059 non-null   float64
 9   a3      8059 non-null   float64
 10  a4      8059 non-null   float64
 11  b1      8059 non-null   float64
 12  b2      8059 non-null   float64
 13  b3      8059 non-null   float64
 14  b4      8059 non-null   float64
 15  c1      8059 non-null   float64
 16  c2      8059 non-null   float64
 17  c3      8059 non-null   float64
 18  c4      8059 non-null   float64
 19  g1      8059 non-null   float64
 20  g2      8059 non-null   float64
 21  l1      8059 non-null   float64
 22  

In [ ]:
df_train.select_dtypes(include=['int64','float64']).describe()

n             p  ...  cent_price_cor  cent_trans_cor
count  11940.000000  11940.000000  ...    11940.000000    11940.000000
mean     548.845059      0.700687  ...       -0.202591        0.362586
std      258.342510      0.122642  ...        0.116617        0.115221
min      100.000000      0.500000  ...       -0.613000       -0.399000
25%      322.000000      0.600000  ...       -0.284000        0.294000
50%      551.000000      0.700000  ...       -0.205000        0.372000
75%      770.000000      0.800000  ...       -0.122000        0.443000
max     1000.000000      0.900000  ...        0.266000        0.717000

[8 rows x 28 columns]

In [ ]:
df_test.select_dtypes(include=['int64','float64']).describe()

id            n  ...           e1           e2
count  8059.000000  8059.000000  ...  8059.000000  8059.000000
mean   4030.000000   549.207842  ...     1.545812     1.556744
std    2326.577243   258.410608  ...     0.838634     0.835856
min       1.000000   100.000000  ...     0.100000     0.100000
25%    2015.500000   326.000000  ...     0.800000     0.800000
50%    4030.000000   551.000000  ...     1.500000     1.600000
75%    6044.500000   772.000000  ...     2.300000     2.300000
max    8059.000000  1000.000000  ...     3.000000     3.000000

[8 rows x 27 columns]

### 2.3 Data Exploration

In [ ]:
df_train.head()

n    p    f   x   y  ...   l4   e1   e2  cent_price_cor  cent_trans_cor
0  709  0.7  0.2  16  12  ...  1.6  1.5  2.6          -0.169           0.375
1  921  0.6  0.3  19  17  ...  1.8  0.6  0.4          -0.075           0.234
2  177  0.8  0.4  14  12  ...  0.6  2.7  0.3          -0.177           0.389
3  415  0.7  0.5  24  11  ...  1.1  0.5  0.1          -0.102           0.358
4  802  0.8  0.4  21  10  ...  1.1  2.0  0.6          -0.034           0.180

[5 rows x 28 columns]

Ideias

se o coeficiente de aleatoriedade for muito grande e os coef de valor de mercado e valor de mercado e valor fundamental forem baixos, entao os lances serão loucos




Correlação. Os atributos tem quase nenhuma correlação entre si, apenas com os targets

In [ ]:
corr = df_train.corr()
corr = corr.style.background_gradient(cmap='Blues')
corr

In [82]:
import plotly.express as px

Usando as distribuições da pra ver que os dados estão em torno da média, que é a situação ideal

In [ ]:
px.histogram(df_train, x = 'cent_price_cor')

In [ ]:
px.violin(df_train, y='cent_price_cor', box = True, points = 'all')

Os dados mostram que a distribuição está em torno da média mas tem alguns outliers na calda da distribuição

In [141]:
px.histogram(df_train, x = 'cent_trans_cor')

In [142]:
px.violin(df_train, y = 'cent_trans_cor', box = True, points = 'all')

Temos classes balenceadas no número de traders. Provavelmente o outlier anterior é esse número de players igual a 1000

In [143]:
px.histogram(df_train, x = 'n')

In [83]:
px.histogram(df_train, x = 'xyz')

In [85]:
px.violin(df_train, y='xyz', box = True, points = 'all')

In [86]:
df_train['xyz'] = np.log(df_train['xyz'])

In [87]:
px.histogram(df_train, x = 'xyz')

In [88]:
px.violin(df_train, y='xyz', box = True, points = 'all')

In [ ]:
px.histogram(df_train, x = 'cent_trans_cor')
px.violin(df_train, y='cent_price_cor', box = True, points = 'all')

In [89]:
df_train.head()

n    p    f   x   y  ...   e1   e2  cent_price_cor  cent_trans_cor       xyz
0  709  0.7  0.2  16  12  ...  1.5  2.6          -0.169           0.375  7.203406
1  921  0.6  0.3  19  17  ...  0.6  0.4          -0.075           0.234  8.080237
2  177  0.8  0.4  14  12  ...  2.7  0.3          -0.177           0.389  6.733402
3  415  0.7  0.5  24  11  ...  0.5  0.1          -0.102           0.358  6.269096
4  802  0.8  0.4  21  10  ...  2.0  0.6          -0.034           0.180  6.445720

[5 rows x 29 columns]


```
df = px.data.iris()
fig = px.parallel_coordinates(df, color="species_id", labels={"species_id": "Species",
                  "sepal_width": "Sepal Width", "sepal_length": "Sepal Length",
                  "petal_width": "Petal Width", "petal_length": "Petal Length", },
                    color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=2)
fig.show()

```



## 3 Data Preparation

Além de juntar o preço fundamental + a agressividade, eu pensei em juntar o preço do mercado + a desvalorização para os consumidores/traders
já que o preço do mercado tá relacionado a desvalorização

## 4 Metrics

## 5 Modeling

In [300]:
df_train.head(2)

n    p    f   x   y  ...   e1   e2  cent_price_cor  cent_trans_cor   aaa
0  709  0.7  0.2  16  12  ...  1.5  2.6          -0.169           0.375 -0.22
1  921  0.6  0.3  19  17  ...  0.6  0.4          -0.075           0.234  0.06

[2 rows x 29 columns]

### 5.0 Utils

Achatando os dados para facilitar o treinamento usando normalização

In [1]:
def normalize_dataset(df_train, df_test):
  #Instanciando o normalizador
  scaler = StandardScaler()
  #Treinando com os dados
  scaler.fit(df_train)
  #Normalizando as features
  df_train = scaler.transform(df_train)
  df_test  = scaler.transform(df_test)
  return  df_train, df_test

Separa o dataframe que temos os alvos em train e test. Fazemos o split pra as correlações de transação e preço em relação a centralidade

In [2]:
def split_dataset(X, y):
  #Split, separa em dados de treino e teste, sendo 80% para treino
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
  return X_train, X_test, y_train, y_test

Constrói o modelo com 2 saídas para cada atributo a ser previsto

RMSProp está causando overfitting no modelo, desempenho com o Adam melhorou demais

In [249]:
def build_model(X):
  n_input = X.shape[1]
  model = tf.keras.Sequential([
                            tf.keras.Input(shape=(n_input,)),
                            tf.keras.layers.Dense(n_input, kernel_initializer = tf.keras.initializers.HeNormal(), kernel_regularizer = tf.keras.regularizers.l2(l=0.01)),                                                        
                            tf.keras.layers.Dropout(0.25),                            
                            tf.keras.layers.Activation(tf.nn.relu),
                            tf.keras.layers.Dense(2)
  ])

  #optimizer = tf.keras.optimizers.RMSprop(0.001)
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss = 'mae',
                  optimizer = optimizer,
                  metrics = ['mae', 'mse'])
  return model

In [196]:
def build_model(X_train):
  n_input = X_train.shape[1]
  model = tf.keras.Sequential([
                            tf.keras.Input(shape=(n_input,)),
                            tf.keras.layers.Dense(48, kernel_initializer = tf.keras.initializers.HeNormal(), kernel_regularizer = tf.keras.regularizers.l2(l=0.01)),                            
                            tf.keras.layers.Dropout(0.25),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Activation(tf.nn.relu),
                            tf.keras.layers.Dense(48, kernel_initializer = tf.keras.initializers.HeNormal(), kernel_regularizer = tf.keras.regularizers.l2(l=0.01)),                            
                            tf.keras.layers.Dropout(0.25),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Activation(tf.nn.relu),
                            tf.keras.layers.Dense(48, kernel_initializer = tf.keras.initializers.HeNormal(), kernel_regularizer = tf.keras.regularizers.l2(l=0.01)),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dropout(0.25),
                            tf.keras.layers.Activation(tf.nn.relu),
                            tf.keras.layers.Dense(2)
  ])

  #optimizer = tf.keras.optimizers.RMSprop(0.001)
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss = 'mae',
                  optimizer = optimizer,
                  metrics = ['mae', 'mse'])
  return model

Função de treino, feita para avaliação

In [233]:
def train(X_train, X_test, y_train, y_test):
  model = build_model(X_train)
  history = model.fit(x = X_train,
                      y = y_train,
                      batch_size = 32,
                      epochs = 30,
                      validation_data = (X_test, y_test),            
                      verbose = 1)
  return model

Função de treino pra treinar com todos os dados do desafio

In [222]:
def final_train(X, y):
  model = build_model(X)
  history = model.fit(x = X,
                      y = y,
                      batch_size = 32,
                      epochs = 100,
                      verbose = 1,
                      callbacks = [tf.keras.callbacks.EarlyStopping(monitor='mae', mode='max', patience = 20)])
  return model

### 5.2 Initial Tests

In [173]:
df_train.head()

a1   a3   b1   b3   c1  ...     l3   e1   e2  cent_price_cor  cent_trans_cor
0  3.8  2.3  1.4  3.1  4.7  ...  0.089  1.5  2.6          -0.169           0.375
1  3.0  2.4  2.0  2.3  3.9  ...  0.076  0.6  0.4          -0.075           0.234
2  3.2  1.8  2.5  1.8  4.4  ...  0.073  2.7  0.3          -0.177           0.389
3  1.3  1.5  3.1  3.6  2.6  ...  0.022  0.5  0.1          -0.102           0.358
4  4.4  2.6  2.2  1.4  4.3  ...  0.046  2.0  0.6          -0.034           0.180

[5 rows x 15 columns]

In [174]:
df_test.head(2)

id   a1   a3   b1   b3   c1   c3    c4     g1   g2     l1     l3   e1   e2
0   1  1.0  1.3  3.7  2.5  4.2  3.1  0.27  0.198  1.8  0.149  0.078  0.5  2.9
1   2  2.2  1.6  4.7  4.9  4.9  2.2  0.16  0.172  1.0  0.086  0.030  0.2  1.6

Aplicando normalização

In [238]:
data_train, data_test = normalize_dataset(df_train.drop(columns=['cent_price_cor','cent_trans_cor']), df_test.drop(columns=['id']))

Separando os dados em treino e teste

In [232]:
X_train, X_test, y_train, y_test = split_dataset(data_train, df_train[['cent_price_cor','cent_trans_cor']])

In [243]:
model = train(X_train, X_test, y_train, y_test)

Epoch 1/30
299/299 [==============================] - 1s 3ms/step - loss: 1.7447 - mae: 0.8137 - mse: 1.1463 - val_loss: 0.6258 - val_mae: 0.1817 - val_mse: 0.0521
Epoch 2/30
299/299 [==============================] - 1s 2ms/step - loss: 0.5425 - mae: 0.2140 - mse: 0.0772 - val_loss: 0.1736 - val_mae: 0.0955 - val_mse: 0.0144
Epoch 3/30
299/299 [==============================] - 1s 2ms/step - loss: 0.1460 - mae: 0.0983 - mse: 0.0155 - val_loss: 0.0977 - val_mae: 0.0912 - val_mse: 0.0132
Epoch 4/30
299/299 [==============================] - 1s 2ms/step - loss: 0.0975 - mae: 0.0932 - mse: 0.0138 - val_loss: 0.0913 - val_mae: 0.0902 - val_mse: 0.0130
Epoch 5/30
299/299 [==============================] - 1s 2ms/step - loss: 0.0938 - mae: 0.0929 - mse: 0.0138 - val_loss: 0.0908 - val_mae: 0.0904 - val_mse: 0.0130
Epoch 6/30
299/299 [==============================] - 1s 2ms/step - loss: 0.0924 - mae: 0.0920 - mse: 0.0135 - val_loss: 0.0905 - val_mae: 0.0902 - val_mse: 0.0129
Epoch 7/30
299/2

### 5.2 Implementação

Com os dados normalizados na seção anterior, vamos treinar o modelo com todos os dados de treino

In [250]:
model = final_train(data_train, df_train[['cent_price_cor','cent_trans_cor']])

Epoch 1/50
374/374 [==============================] - 1s 2ms/step - loss: 0.9819 - mae: 0.5988 - mse: 0.6810
Epoch 2/50
374/374 [==============================] - 1s 2ms/step - loss: 0.2530 - mae: 0.1398 - mse: 0.0323
Epoch 3/50
374/374 [==============================] - 1s 2ms/step - loss: 0.1284 - mae: 0.0964 - mse: 0.0148
Epoch 4/50
374/374 [==============================] - 1s 2ms/step - loss: 0.1011 - mae: 0.0926 - mse: 0.0137
Epoch 5/50
374/374 [==============================] - 1s 1ms/step - loss: 0.0938 - mae: 0.0917 - mse: 0.0134
Epoch 6/50
374/374 [==============================] - 1s 2ms/step - loss: 0.0919 - mae: 0.0914 - mse: 0.0134
Epoch 7/50
374/374 [==============================] - 1s 1ms/step - loss: 0.0916 - mae: 0.0914 - mse: 0.0133
Epoch 8/50
374/374 [==============================] - 1s 2ms/step - loss: 0.0918 - mae: 0.0917 - mse: 0.0133
Epoch 9/50
374/374 [==============================] - 1s 2ms/step - loss: 0.0927 - mae: 0.0926 - mse: 0.0137
Epoch 10/50
374/374

Realizando Predição

In [251]:
results = model.predict(data_test)

Salvando o resultado em dataframe no formato exigido pela competição

In [252]:
predictions = pd.DataFrame(results, columns=['cent_price_cor', 'cent_trans_cor'])
predictions

cent_price_cor  cent_trans_cor
0          -0.209069        0.369781
1          -0.209613        0.370479
2          -0.210747        0.371933
3          -0.208048        0.368472
4          -0.208055        0.368480
...              ...             ...
8054       -0.208975        0.369661
8055       -0.208214        0.368684
8056       -0.208232        0.368708
8057       -0.208111        0.368552
8058       -0.210462        0.371569

[8059 rows x 2 columns]

Exibindo as 20 primeiras linhas da predição

In [253]:
predictions.head(20)

cent_price_cor  cent_trans_cor
0        -0.209069        0.369781
1        -0.209613        0.370479
2        -0.210747        0.371933
3        -0.208048        0.368472
4        -0.208055        0.368480
5        -0.209585        0.370443
6        -0.209171        0.369913
7        -0.208048        0.368472
8        -0.210842        0.372056
9        -0.208048        0.368472
10       -0.209471        0.370298
11       -0.210893        0.372121
12       -0.208048        0.368472
13       -0.208402        0.368925
14       -0.208969        0.369653
15       -0.208048        0.368472
16       -0.208048        0.368472
17       -0.208048        0.368472
18       -0.208048        0.368472
19       -0.208859        0.369512

Persistindo em disco

In [255]:
predictions.to_csv('predictions.csv')

## Modelo campeão

In [ ]:
def build_model(X):
  n_input = X.shape[1]
  model = tf.keras.Sequential([
                            tf.keras.Input(shape=(n_input,)),
                            tf.keras.layers.Dense(n_input, kernel_initializer = tf.keras.initializers.HeNormal(), kernel_regularizer = tf.keras.regularizers.l2(l=0.01)),                                                        
                            tf.keras.layers.Dropout(0.25),                            
                            tf.keras.layers.Activation(tf.nn.relu),
                            tf.keras.layers.Dense(2)
  ])

  #optimizer = tf.keras.optimizers.RMSprop(0.001)
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  #optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss = 'mae',
                  optimizer = optimizer,
                  metrics = ['mae', 'mse'])
  return model

In [ ]:
def final_train(X, y):
  model = build_model(X)
  history = model.fit(x = X,
                      y = y,
                      batch_size = 32,
                      epochs = 50,
                      verbose = 1)
  return model

In [211]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)